In [1]:
using Revise
using Pkg
Pkg.resolve()
Pkg.instantiate()


┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/daniel/.julia/packages/Revise/7QZSP/src/packagedef.jl:67
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Project.toml`
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Manifest.toml`


In [2]:
using Pkg
Pkg.activate("..")

include("../src/ChemQHam.jl")
using .ChemQHam

using HDF5
using TensorKit
using SparseArrays
using LinearAlgebra

using ITensorMPS
using ITensorChemistry
import TensorKit: ⊕


  Activating project at `~/Desktop/Felman/projects/Quantum/ChemQHam`
┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/daniel/.julia/packages/Revise/7QZSP/src/packagedef.jl:67
┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/daniel/.ju

In [3]:
phySpace = genPhySpace("su2")
dataType = Float64

Float64

### Test Hopping term a1c2

In [4]:
#### a1c2 ####

n_sites=2
init_ord = collect(1:n_sites)
algo="Hungarian"
ops_tol= 1e-8
ops_factor = 1e0

a_loc = 1
c_loc = 2

println("Testing a1c2...")

# IT
# it_terms_range = [4,5]
# (; opsum, h1e, h2e, nuc_e) = ITChem_opsum(n_sites; idx_factor_list=nothing, it_terms_range=it_terms_range, ops_tol=ops_tol, ops_factor=ops_factor)

(; opsum, h1e, h2e, nuc_e) = ITChem_opsum(n_sites; idx_factor_list=nothing, it_terms_range=nothing, ops_tol=ops_tol, ops_factor=ops_factor)
aacc_sites = map(op->[local_op.sites[1] for local_op in op.args[2]], opsum)
it_terms_range = findall(sites->length(sites)==2 && sites[1] == a_loc && sites[2] == c_loc, aacc_sites)
opsum = reduce(+, op for op in opsum[it_terms_range]) * ops_factor

println("opsum: ", opsum)


# TK

terms = gen_ChemOpSum(h1e, h2e, nuc_e, init_ord; tol=ops_tol, spin_symm=true)

terms_range = findall(t->length(t.sites)==2 && t.sites[1] == a_loc && t.sites[2] == c_loc , terms)

op_terms=terms[terms_range] * ops_factor
println("op_terms: ", op_terms)


Testing a1c2...
opsum: sum(
  0.1479263190288496 c†↑(1,) c↑(2,)
  0.1479263190288496 c†↓(1,) c↓(2,)
)
op_terms: ChemOpSum with 1 terms:
  1. [ a1 ]_1 ── [ c1 ]_2    factor = 0.1479263190288496


In [6]:

table, primary_ops, factors = terms_to_table(n_sites, op_terms)
symbolic_mpo, virt_spaces = construct_symbolic_mpo(table, primary_ops, factors; algo="Hungarian", verbose=false)
mpo = symbolic_to_tensorkit_mpo(symbolic_mpo, virt_spaces, phySpace; spin_symm=false, dataType=dataType, verbose=false)
tk_mat = mpo_to_mat(mpo)
tk_nz = collect(zip(findnz(tk_mat)...))


# it_mat = ITensor_mat(chem_data, init_ord; ops_tol=ops_tol, ops_factor=ops_factor, opsum=opsum)
it_mat = ITChem_mat(opsum, n_sites)
it_nz = collect(zip(findnz(it_mat)...))

# compare
cond_a1c2 = isapprox(norm(tk_mat-it_mat), 0.0, atol=1e-6)
println("Condition a1c2: ", cond_a1c2)


┌ Warning: FermionParity Arrays do not preserve categorical properties.
└ @ TensorKitSectors /home/daniel/.julia/packages/TensorKitSectors/8zBl2/src/fermions.jl:53


Condition a1c2: true
